<a href="https://colab.research.google.com/github/frikel12/Machine-Learning-and-Deep-Learningproject/blob/main/Moroccan_Sentiment_Analysis_using_EmbeddingLayer_with_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Library

In [ ]:
import numpy as np
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf


#Loading the Data

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

path='/content/drive/My Drive/Datasets/marocSA/train.csv'
train = pd.read_csv(path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train.head()

,ID,comment,label
0,1,انا اوصي من هذا المنبر الكل للتوجه الى مراكز ا...,1
1,2,\nهناك الكثير لا يفهم المقصود كورونا ليست صعبة...,1
2,3,الحمد لله ارقام مقبولة مقارنة بدول المنطقة لول...,1
3,4,انا شخصيا أؤيد ما فرضته السلطات من ضرورة الإدل...,1
4,5,\nنفس الشئ في مدينة برشيد مراكز التلقيح مغلقة ...,1


In [ ]:
train.shape

(1920, 3)

In [ ]:
train['label'].unique()

array([1, 0])

#Preprocessing Data

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:

def split_into_words(text):
    words = text.split()
    return words

def remove_punctuation(words):
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = [re_punc.sub(' ', w) for w in words]
    return stripped

def remove_stopwords(words):
    stop_words = set(stopwords.words('arabic'))
    stop_words = [word for word in stop_words if word not in ['لا', 'ولا']]
    words = [w for w in words if not w in stop_words]
    return words

def to_sentence(words):
    return ' '.join(words)

def denoise_text(text):
    words = split_into_words(text)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return to_sentence(words)


In [ ]:
train['cleaned_text'] = train['comment'].apply(denoise_text)
train['cleaned_text']=train.cleaned_text.str.replace('\n|#','')
train.head()

<ipython-input-10-b4b42714aff1>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train['cleaned_text']=train.cleaned_text.str.replace('\n|#','')


,ID,comment,label,cleaned_text
0,1,انا اوصي من هذا المنبر الكل للتوجه الى مراكز ا...,1,انا اوصي المنبر الكل للتوجه الى مراكز التلقيح ...
1,2,\nهناك الكثير لا يفهم المقصود كورونا ليست صعبة...,1,الكثير لا يفهم المقصود كورونا صعبة الشباب لكنه...
2,3,الحمد لله ارقام مقبولة مقارنة بدول المنطقة لول...,1,الحمد لله ارقام مقبولة مقارنة بدول المنطقة ظهو...
3,4,انا شخصيا أؤيد ما فرضته السلطات من ضرورة الإدل...,1,انا شخصيا أؤيد فرضته السلطات ضرورة الإدلاء بجو...
4,5,\nنفس الشئ في مدينة برشيد مراكز التلقيح مغلقة ...,1,الشئ مدينة برشيد مراكز التلقيح مغلقة غاية الاث...


In [ ]:
print(train['comment'][1])
print(train['cleaned_text'][1])

X_data = train['cleaned_text']
y_data = np.array(train['label'])


هناك الكثير لا يفهم المقصود كورونا ليست صعبة على الشباب لكنها صعبة على اصحاب مرض مزمن كالسكري …. لهذا الدولة اختارت ان تنقد امهاتنا و ابائنا و جميع مرضى السكري و من بعد ذلك قام قبل سيعم الجميع هو في الحقيقة عندما يتم التلقيح و يتم محاصرة الفيروس بالتلقيح ينتهي المشكل قبل أن يعم الفيروس كل الشرائح لهذا نتمنى ان تسرع الدولة في مخططها هذا لما له من تأثير على محاربة الوباء في اقرب وقت ممكن نتمنى من الله. التوفيق لكل من يساهم من بعيد او قريب على انجاح هذه المهمة و الشكر كذلك لصاحب الجلالة الذي لا يبخل في ان يكون هو السابق لمثل هذه المبادرة التي ستعود عليه بالصحة و العافية و تحية للشعب المغربي من طنجة إلى الݣرݣرات

الكثير لا يفهم المقصود كورونا صعبة الشباب لكنها صعبة اصحاب مرض مزمن كالسكري …  لهذا الدولة اختارت ان تنقد امهاتنا ابائنا مرضى السكري سيعم الجميع الحقيقة عندما يتم التلقيح يتم محاصرة الفيروس بالتلقيح ينتهي المشكل يعم الفيروس الشرائح لهذا نتمنى ان تسرع الدولة مخططها تأثير محاربة الوباء اقرب وقت ممكن نتمنى الله  التوفيق لكل يساهم بعيد او قريب انجاح المهمة الشكر لصاحب الجلالة لا يبخل

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X_data, y_data, test_size=0.1)

# ANN with Embedding

In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token = '<OOV>')
tokenizer.fit_on_texts(X_data)
word_index = tokenizer.word_index
VOCAB_SIZE = len(word_index)+1
VOCAB_SIZE

19552

In [ ]:
#maxlen = max([len(x) for x in X_train])
maxlen = 128

In [ ]:
data_sequences = tokenizer.texts_to_sequences(X_data)
data_padded_sequences = pad_sequences(data_sequences,maxlen=maxlen,padding='post',truncating='post')


In [ ]:
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded_sequences = pad_sequences(train_sequences,maxlen=maxlen,padding='post',truncating='post')

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded_sequences = pad_sequences(test_sequences,maxlen=maxlen,padding='post',truncating='post')

In [ ]:
len(train_padded_sequences)

1728

In [ ]:
embedding_dim = 200

In [ ]:
# Create a Sequential model
model = tf.keras.Sequential()

# Add an Embedding layer
model.add(tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=embedding_dim, input_length=maxlen))  # max_len is the maximum sequence length
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 128, 200)          3910400   
                                                                 
 flatten (Flatten)           (None, 25600)             0         
                                                                 
 dense (Dense)               (None, 64)                1638464   
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5548929 (21.17 MB)
Trainable params: 5548929 (21.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint('/content/drive/My Drive/Models/ANN_with_Embedding/ANN_Model200_weights_callback.keras', verbose=1, save_best_only=True, save_weights_only=True)
history = model.fit(train_padded_sequences, y_train, validation_data = (test_padded_sequences, y_test), epochs=20, callbacks=[checkpointer])


Epoch 1/20
54/54 [==============================] - ETA: 0s - loss: 0.3087 - accuracy: 0.8906
Epoch 1: val_loss improved from inf to 0.32517, saving model to /content/drive/My Drive/Models/ANN_with_Embedding/ANN_Model200_weights_callback.keras
54/54 [==============================] - 8s 150ms/step - loss: 0.3087 - accuracy: 0.8906 - val_loss: 0.3252 - val_accuracy: 0.8542
Epoch 2/20
54/54 [==============================] - ETA: 0s - loss: 0.1181 - accuracy: 0.9635
Epoch 2: val_loss did not improve from 0.32517
54/54 [==============================] - 7s 133ms/step - loss: 0.1181 - accuracy: 0.9635 - val_loss: 0.3385 - val_accuracy: 0.8438
Epoch 3/20
53/54 [============================>.] - ETA: 0s - loss: 0.0368 - accuracy: 0.9941
Epoch 3: val_loss did not improve from 0.32517
54/54 [==============================] - 5s 91ms/step - loss: 0.0363 - accuracy: 0.9942 - val_loss: 0.3388 - val_accuracy: 0.8438
Epoch 4/20
52/54 [===========================>..] - ETA: 0s - loss: 0.0144 - accur

In [ ]:
loss, acc = model.evaluate(test_padded_sequences, y_test)
print("accuracy: {:5.2f}%".format(100 * acc))

6/6 [==============================] - 0s 4ms/step - loss: 0.5172 - accuracy: 0.8490
accuracy: 84.90%


In [ ]:
model.save_weights('/content/drive/My Drive/Models/ANN_with_Embedding/ANN_Model200_weights.keras')

# Loading Model

In [ ]:
embedding_dim = 200

# Create a Sequential model
model = tf.keras.Sequential()

# Add an Embedding layer
model.add(tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=embedding_dim, input_length=maxlen))  # max_len is the maximum sequence length
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 128, 200)          3910400   
                                                                 
 flatten_1 (Flatten)         (None, 25600)             0         
                                                                 
 dense_2 (Dense)             (None, 64)                1638464   
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5548929 (21.17 MB)
Trainable params: 5548929 (21.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.load_weights('/content/drive/My Drive/Models/ANN_with_Embedding/ANN_Model200_weights_callback.keras')

loss, acc = model.evaluate(test_padded_sequences, y_test)
print("accuracy: {:5.2f}%".format(100 * acc))

6/6 [==============================] - 0s 4ms/step - loss: 0.3252 - accuracy: 0.8542
accuracy: 85.42%


# Submission

In [ ]:
test_stage = pd.read_csv('/content/drive/My Drive/Datasets/marocSA/test_stage1.csv')

sample_submission = pd.read_csv('/content/drive/My Drive/Datasets/marocSA/sample_submission.csv')

In [ ]:
sample_submission.head()

,ID,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [ ]:
test_stage['cleaned_text'] = test_stage['comment'].apply(denoise_text)
test_stage['cleaned_text']=test_stage.cleaned_text.str.replace('\n|#','')
submission = test_stage['cleaned_text']

submission_sequences = tokenizer.texts_to_sequences(submission)
submission_padded_sequences = pad_sequences(submission_sequences,maxlen=maxlen,padding='post',truncating='post')

predictions = model.predict(submission_padded_sequences)

binary_predictions = (predictions > 0.5).astype(int)
flattened_predictions = binary_predictions.flatten()

sample_submission['label'] = flattened_predictions


8/8 [==============================] - 0s 2ms/step


<ipython-input-20-53557b6b1991>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test_stage['cleaned_text']=test_stage.cleaned_text.str.replace('\n|#','')


In [ ]:
sample_submission.to_csv("/content/drive/My Drive/Models/Arabic_Bert_Model/submission.csv", index=False)